# Cointab data analyst test

Client X tied with multiple courier company
Charges depend on
Weight of the product and distance between warehouse and customer delivery address(Zones)

We need to verify the charges levied by the partners per order are correct as per delivery.


#### Company X Order Report  
Columns: 
ExternOrderNo, 
SKU,
Order Qty.

#### Company X - SKU Master (Gross weight of each product)
Columns: 
SKU
Weight (g)

#### Company X - Pincode Zones 
Columns:
Warehouse Pincode
Customer Pincode
Zone


#### Courier Company – Invoice
Columns:
Order ID
AWB Code
Charged Weight
Warehouse Pincode
Customer Pincode
Zone
Type of Shipment

#### Courier Company – Rates
Given the Rates based on different slab weights, zones, and Type of shipment

#### Expected_Result

Order ID - in Company order report

AWB number - in Courier Company Invoice

Total weight as per X (KG) = Calculated based on weights for differet orders and their Quantity ( df4['new_weight'] = df4["Order Qty"]* 0.001* df4["Weight (g)"] )

Weight slab as per X (KG) = round off by celing at 0.5 (lambda x: (math.ceil(x*2)/2) )

Total weight as per Courier Company (KG) = in Courier Company Invoice

Weight slab charged by Courier Company (KG) = round off by celing at 0.5 (lambda x: (math.ceil(x*2)/2) )

Delivery Zone as per X = in  Company X - Pincode Zones 

Delivery Zone charged by Courier Company = in Courier Company – Invoice

Expected Charge as per X (Rs.) = 

- Calculted based on weather the type of shipment in forword charges  or Foreord and rto charges
- after finding that find wheater it comes under the slab 0.5 if so use frw_fixed or else use frw_fixed for first 0.5kg and frw_additional for remaining kgs. then based on the zone,map the rate and calculte

Difference Between Expected Charges and Billed Charges (Rs.) = 

- calucated by finding the difference between calculated and billed


#### Summery 

We must Find and Conclude that the total orders are correctly charged, over charged , or under charged


In [48]:
import math
import pandas as pd

### Reading files and renaming columns

In [49]:
Company_X_Orders  = pd.read_excel("Company X - Order Report.xlsx")
Company_X_Orders .head()

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0


In [50]:
Company_Pincode = pd.read_excel("Company X - Pincode Zones.xlsx")
Company_Pincode.rename(columns = {'Zone': 'Delivery Zone as per X'},inplace=True)
Company_Pincode = Company_Pincode['Delivery Zone as per X']

In [51]:
Courier_company_invoice = pd.read_excel("Courier Company - Invoice.xlsx")
Courier_company_invoice.drop(['Warehouse Pincode','Customer Pincode'],axis=1,inplace=True)
Courier_company_invoice.columns  = ['AWB Number', 'Order ID', 'Total weight as per Courier Company (KG)','Delivery Zone charged by Courier Company', 'Type of Shipment', 'Charges Billed by Courier Company (Rs.) ']
Courier_company_invoice.head()

,AWB Number,Order ID,Total weight as per Courier Company (KG),Delivery Zone charged by Courier Company,Type of Shipment,Charges Billed by Courier Company (Rs.)
0,1091117222124,2001806232,1.30,d,Forward charges,135.0
1,1091117222194,2001806273,1.00,d,Forward charges,90.2
2,1091117222931,2001806408,2.50,d,Forward charges,224.6
3,1091117223244,2001806458,1.00,b,Forward charges,61.3
4,1091117229345,2001807012,0.15,d,Forward charges,45.4


In [52]:
SKU_Master = pd.read_excel("Company X - SKU Master.xlsx")
SKU_Master.head()

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120


In [53]:
Courier_company_Rates = pd.read_excel('Courier Company - Rates.xlsx')

### Joining Dataframes

##### Calculating Total weight as per X (KG) and Weight slab as per X (KG)

In [54]:
df4 = pd.merge(Company_X_Orders ,SKU_Master,on='SKU')
df4['new_weight'] = df4["Order Qty"]* 0.001* df4["Weight (g)"]

In [55]:
df4

,ExternOrderNo,SKU,Order Qty,Weight (g),new_weight
0,2001827036,8904223818706,1.0,127,0.127
1,2001821995,8904223818706,1.0,127,0.127
2,2001819252,8904223818706,1.0,127,0.127
3,2001816996,8904223818706,1.0,127,0.127
4,2001814580,8904223818706,1.0,127,0.127
...,...,...,...,...,...
396,2001806616,8904223819123,1.0,250,0.250
397,2001806567,8904223815804,1.0,160,0.160
398,2001806567,8904223818577,1.0,150,0.150
399,2001806408,8904223819437,2.0,552,1.104


In [56]:
temp_df4 = df4[["ExternOrderNo","new_weight"]].groupby('ExternOrderNo').sum()
temp_df4.reset_index(level=0, inplace=True)
temp_df4

,ExternOrderNo,new_weight
0,2001806210,0.220
1,2001806226,0.480
2,2001806229,0.500
3,2001806232,1.302
4,2001806233,0.245
...,...,...
119,2001821995,0.477
120,2001822466,1.376
121,2001823564,0.672
122,2001825261,1.557


In [57]:
temp_df4['new_weight'] = temp_df4['new_weight'].round(1)
temp_df4

,ExternOrderNo,new_weight
0,2001806210,0.2
1,2001806226,0.5
2,2001806229,0.5
3,2001806232,1.3
4,2001806233,0.2
...,...,...
119,2001821995,0.5
120,2001822466,1.4
121,2001823564,0.7
122,2001825261,1.6


In [58]:
temp_df4['slab_weight'] = temp_df4['new_weight'].apply(lambda x: (math.ceil(x*2)/2) )

In [59]:
temp_df4.columns = ['Order ID','Total weight as per X (KG)','Weight slab as per X (KG)']
temp_df4

,Order ID,Total weight as per X (KG),Weight slab as per X (KG)
0,2001806210,0.2,0.5
1,2001806226,0.5,0.5
2,2001806229,0.5,0.5
3,2001806232,1.3,1.5
4,2001806233,0.2,0.5
...,...,...,...
119,2001821995,0.5,0.5
120,2001822466,1.4,1.5
121,2001823564,0.7,1.0
122,2001825261,1.6,2.0


###### Joing courier company invoice with founded weight slab

In [60]:
temp_df4 = pd.merge(Courier_company_invoice,temp_df4,on='Order ID')

In [61]:
temp_df4 = pd.concat([temp_df4,Company_Pincode],axis=1)

##### Finding the Expected Charge as per X (Rs.) 

In [62]:
c = []
for i,j,k in zip(temp_df4['Type of Shipment'],temp_df4['Delivery Zone as per X'],temp_df4['Weight slab as per X (KG)']):
    if i == 'Forward charges':
        if k > 0.5:
            c.append(round((Courier_company_Rates['fwd_'+j+'_fixed'][0]) + ((k-0.5)/0.5 * (Courier_company_Rates['fwd_'+j+'_additional'][0])),1))
        else:
            c.append(round(Courier_company_Rates['fwd_'+j+'_fixed'][0]))
    if i == 'Forward and RTO charges':
        if k> 0.5:
            c.append(round((Courier_company_Rates['rto_'+j+'_fixed'][0]) + ((k-0.5)/0.5 * (Courier_company_Rates['rto_'+j+'_additional'][0])),1))
        else:
            c.append(round(Courier_company_Rates['rto_'+j+'_fixed'][0]))
                     

In [63]:
Expected = pd.DataFrame(c)
Expected.columns = ['Expected Charge as per X (Rs.)']

In [64]:
Final_df = pd.concat([temp_df4,Expected],axis=1)
Final_df['Weight slab charged by Courier Company (KG)'] = Final_df['Total weight as per Courier Company (KG)'].apply(lambda x: (math.ceil(x*2)/2) )

In [65]:
Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] = Final_df['Expected Charge as per X (Rs.)'] - Final_df['Charges Billed by Courier Company (Rs.) ']

In [66]:
col_rear = pd.read_excel('Expected_Result.xlsx',sheet_name='Calculations').columns
col_rear

Index(['Order ID', 'AWB Number', 'Total weight as per X (KG)',
       'Weight slab as per X (KG)', 'Total weight as per Courier Company (KG)',
       'Weight slab charged by Courier Company (KG)', 'Delivery Zone as per X',
       'Delivery Zone charged by Courier Company',
       'Expected Charge as per X (Rs.)',
       'Charges Billed by Courier Company (Rs.) ',
       'Difference Between Expected Charges and Billed Charges (Rs.)'],
      dtype='object')

In [67]:
Final_df = Final_df[col_rear]

In [68]:
Final_df

,Order ID,AWB Number,Total weight as per X (KG),Weight slab as per X (KG),Total weight as per Courier Company (KG),Weight slab charged by Courier Company (KG),Delivery Zone as per X,Delivery Zone charged by Courier Company,Expected Charge as per X (Rs.),Charges Billed by Courier Company (Rs.),Difference Between Expected Charges and Billed Charges (Rs.)
0,2001806232,1091117222124,1.3,1.5,1.30,1.5,d,d,135.0,135.0,0.0
1,2001806273,1091117222194,0.6,1.0,1.00,1.0,d,d,90.2,90.2,0.0
2,2001806408,1091117222931,2.3,2.5,2.50,2.5,d,d,224.6,224.6,0.0
3,2001806458,1091117223244,0.7,1.0,1.00,1.0,b,b,61.3,61.3,0.0
4,2001807012,1091117229345,0.2,0.5,0.15,0.5,d,d,45.0,45.4,-0.4
...,...,...,...,...,...,...,...,...,...,...,...
119,2001812941,1091118551656,0.5,0.5,0.73,1.0,b,d,33.0,90.2,-57.2
120,2001809383,1091117614452,0.6,1.0,0.50,0.5,b,d,48.8,86.7,-37.9
121,2001820978,1091120922803,0.5,0.5,0.50,0.5,b,d,33.0,45.4,-12.4
122,2001811475,1091121844806,0.7,1.0,0.50,0.5,e,b,112.1,33.0,79.1


In [69]:
correct_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']==0].count()
correct_amt = Final_df['Charges Billed by Courier Company (Rs.) '][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']==0].sum()
print(correct_count,correct_amt)

20 1958.3


In [70]:
Over_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] < 0].count()
Over_amt = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)']< 0].sum()
print(Over_amt,Over_count)

-5801.2 98


In [71]:
under_count = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0].count()
under_amt = Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'][Final_df['Difference Between Expected Charges and Billed Charges (Rs.)'] > 0].sum()
print(under_amt,under_count)

375.29999999999995 6


In [72]:
dic = [[correct_count,correct_amt],[Over_count,Over_amt],[under_count,under_amt]]
Summary = pd.DataFrame(dic,index=['Total Orders - Correctly Charged','Total Orders - Over Charged','Total Orders - Under Charged'])

In [73]:
Summary.columns = ['Count','Amount'] 

In [74]:
Summary

,Count,Amount
Total Orders - Correctly Charged,20,1958.3
Total Orders - Over Charged,98,-5801.2
Total Orders - Under Charged,6,375.3


In [75]:
writer = pd.ExcelWriter('Final_result.xlsx',engine='xlsxwriter')   
Summary.to_excel(writer,sheet_name='Summary')
Final_df.to_excel(writer,sheet_name='Calculations',index=False)
writer.save()